In [94]:
%load_ext autoreload

In [95]:
%autoreload 2

In [1]:
%matplotlib inline

In [2]:
from matplotlib import pyplot as plt

### system imports 

In [3]:
import pathlib
import sys
import os 
import time 
import pickle
import time 
from datetime import timedelta

### get current directory 


In [4]:
CWD = pathlib.Path.cwd()

In [5]:
HOME = pathlib.Path.home()

### add the path to the custom functions

In [96]:
sys.path.append(str(HOME.joinpath('research/dl4seas/dl4seas/')))

In [97]:
from NN import *
from utils import *

### Scientific Stack 

In [8]:
import numpy as np 
import pandas as pd 
import xarray as xr

### Tensorflow and Keras 

In [9]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

### set the random seeds for reproducible results 

In [10]:
np.random.seed(42)
tf.random.set_seed(42)

### check if the GPU is available 

In [11]:
if len(tf.config.list_physical_devices('GPU')) >= 1: 
    compute = 'GPU'
else: 
    compute = 'CPU'

In [12]:
compute

'GPU'

### Loads and prepare the data 

In [26]:
dpath_vcsn = '/media/nicolasf/END19101/data/VCSN/'
dpath_gcm = '/media/nicolasf/END19101/data/GCMs/processed/hindcasts/CDS/ECMWF/T2M/'

### target (VCSN)

In [16]:
var_name = "TMEAN"

In [27]:
if "TMEAN" in var_name:
    dpath = pathlib.Path(f"{dpath_vcsn}/{var_name.replace('_N','')}/seasonal_anomalies_VCSN_{var_name}_N.nc")
elif "RAIN" in var_name:
    dpath = pathlib.Path(f"{dpath_vcsn}/{var_name.replace('_BC','')}/seasonal_anomalies_VCSN_{var_name}_BC.nc")

In [28]:
vcsn = xr.open_dataset(dpath) 

In [29]:
vcsn

<xarray.Dataset>
Dimensions:         (lat: 257, lon: 241, time: 490)
Coordinates:
  * lat             (lat) float32 -47.275 -47.225 -47.175 ... -34.475 -34.425
  * lon             (lon) float32 166.475 166.525 166.575 ... 178.425 178.475
  * time            (time) datetime64[ns] 1979-03-31 1979-04-30 ... 2019-12-31
    month           (time) int64 ...
Data variables:
    agent           (time, lat, lon) float32 ...
    Tmean_N         (time, lat, lon) float32 ...
    Tmean_N_interp  (time, lat, lon) float64 ...
    mask            (lat, lon) float64 ...

In [30]:
vcsn = vcsn[['Tmean_N_interp']]

In [31]:
vcsn

<xarray.Dataset>
Dimensions:         (lat: 257, lon: 241, time: 490)
Coordinates:
  * lat             (lat) float32 -47.275 -47.225 -47.175 ... -34.475 -34.425
  * lon             (lon) float32 166.475 166.525 166.575 ... 178.425 178.475
  * time            (time) datetime64[ns] 1979-03-31 1979-04-30 ... 2019-12-31
    month           (time) int64 3 4 5 6 7 8 9 10 11 12 ... 4 5 6 7 8 9 10 11 12
Data variables:
    Tmean_N_interp  (time, lat, lon) float64 ...

In [32]:
lfiles = list_files(dpath_gcm, pattern="ECMWF_T2M_seasonal_anomalies", extension=".nc", exclude='interp', verbose=1)

loaded files, list length 288
the first file is /media/nicolasf/END19101/data/GCMs/processed/hindcasts/CDS/ECMWF/T2M/ECMWF_T2M_seasonal_anomalies_1993_01.nc
the last file is /media/nicolasf/END19101/data/GCMs/processed/hindcasts/CDS/ECMWF/T2M/ECMWF_T2M_seasonal_anomalies_2016_12.nc


### read the multiple files data set using xarray and dask, not that it will not be loaded in RAM

In [33]:
gcm = xr.open_mfdataset(lfiles, concat_dim='time', combine='nested', parallel=True)

In [34]:
gcm.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:     (lat: 181, lon: 360, member: 25, step: 3, time: 288)
Coordinates:
    surface     int64 0
  * lat         (lat) float64 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon         (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * member      (member) int64 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23 24
  * step        (step) int64 3 4 5
  * time        (time) datetime64[ns] 1993-01-01 1993-02-01 ... 2016-12-01
    valid_time  (time, step) datetime64[ns] dask.array<chunksize=(1, 3), meta=np.ndarray>
    month       (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 3 4 5 6 7 8 9 10 11 12
Data variables:
    t2m         (time, member, step, lat, lon) float32 dask.array<chunksize=(1, 25, 3, 181, 360), meta=np.ndarray>>

### selects the training set 

In [36]:
gcm_train = gcm.sel(time=slice('1993','2010'))[['t2m']]

### selects the validation set 

In [37]:
gcm_val = gcm.sel(time=slice('2011','2015'))[['t2m']]

### keep one year for testing 

In [38]:
gcm_test = gcm.sel(time='2016')[['t2m']]

**prepare the data**: 
    
    1) select the step (leadtime)
    2) to increase the size of the training and validation sets, consider each member as an instance (stacking along the time and member dimension)
    3) transpose to have the instances as the first dimension

In [39]:
def prep_data(ds): 
    ds = ds[['t2m']].sel(step=3)
    ds = ds.stack(instance=('time','member'))
    ds = ds.transpose('instance','lat','lon')
    return ds

In [41]:
gcm_train, gcm_val, gcm_test = map(prep_data, [gcm_train, gcm_val, gcm_test])

In [112]:
rdatetimes_train = gcm_train.indexes["instance"].get_level_values(0)

In [113]:
rdatetimes_val = gcm_val.indexes["instance"].get_level_values(0)

In [114]:
rdatetimes_test = gcm_test.indexes["instance"].get_level_values(0)

### shift the VCSN time 

In [106]:
vcsn.time[0]

<xarray.DataArray 'time' ()>
array('1979-03-01T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 1979-03-01
    month    int64 3

In [107]:
vcsn.time[-1]

<xarray.DataArray 'time' ()>
array('2019-12-01T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2019-12-01
    month    int64 12

In [104]:
vcsn['time'] = shift_time(vcsn.time.to_index(), shift='Month Begin')

In [108]:
vcsn.time[0]

<xarray.DataArray 'time' ()>
array('1979-03-01T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 1979-03-01
    month    int64 3

In [109]:
vcsn['time'][-1]

<xarray.DataArray 'time' ()>
array('2019-12-01T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2019-12-01
    month    int64 12

In [116]:
vcsn_train = vcsn.sel(time=rdatetimes_train)

In [117]:
vcsn_val = vcsn.sel(time=rdatetimes_val)

In [118]:
vcsn_test = vcsn.sel(time=rdatetimes_test)

### rename 'time' to 'instance' in vcsn

In [121]:
vcsn_train, vcsn_val, vcsn_test = map(lambda x: x.rename({'time':'instance'}), [vcsn_train, vcsn_val, vcsn_test])

### Now checks if everything is OK 

In [123]:
gcm_train.dims

Frozen(SortedKeysDict({'lat': 181, 'lon': 360, 'instance': 5400}))

In [124]:
vcsn_train.dims

Frozen(SortedKeysDict({'lat': 257, 'lon': 241, 'instance': 5400}))

In [129]:
vcsn

<xarray.Dataset>
Dimensions:         (lat: 257, lon: 241, time: 490)
Coordinates:
  * lat             (lat) float32 -47.275 -47.225 -47.175 ... -34.475 -34.425
  * lon             (lon) float32 166.475 166.525 166.575 ... 178.425 178.475
  * time            (time) datetime64[ns] 1979-03-01 1979-04-01 ... 2019-12-01
    month           (time) int64 3 4 5 6 7 8 9 10 11 12 ... 4 5 6 7 8 9 10 11 12
Data variables:
    Tmean_N_interp  (time, lat, lon) float64 ...

### define the parameters for the model

In [132]:
batch_size = 32

In [138]:
data_train = XrDataGenerator(gcm_train, vcsn_train, {'t2m':None}, 'Tmean_N_interp', \
                             norm=True, batch_size=batch_size, \
                             mean=None, std=None, shuffle=True, load=False)

In [139]:
data_val = XrDataGenerator(gcm_val, vcsn_val, {'t2m':None}, 'Tmean_N_interp', \
                           norm=True, batch_size=batch_size, \
                           mean=data_train.mean, std=data_train.std, shuffle=True, load=False)

In [140]:
data_test = XrDataGenerator(gcm_test, vcsn_test, {'t2m':None}, 'Tmean_N_interp', \
                            norm=True, batch_size=batch_size, \
                            mean=data_train.mean, std=data_train.std, shuffle=True, load=False)

### checks the shapes 

In [141]:
data_train[0][0].shape

(32, 181, 360, 1)

In [143]:
data_train[0][1].shape

(32, 257, 241, 1)

In [154]:
data_val[0][0].shape

(32, 181, 360, 1)

In [156]:
data_val[0][1].shape

(32, 257, 241, 1)

In [157]:
data_test[0][0].shape

(32, 181, 360, 1)

In [158]:
data_test[0][1].shape

(32, 257, 241, 1)

### load the model 

In [13]:
model = keras.models.load_model('./saved_autoencoder_ConvAE_wo_MaxPooling_ECMWF_T2M_run_2020_08_19-14_52_16_20_epochs_GPU_fullCNN/')

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 181, 360, 1)]     0         
_________________________________________________________________
resize_layer (ResizeLayer)   (None, 184, 360, 1)       0         
_________________________________________________________________
periodic_padding2d (Periodic (None, 200, 376, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 200, 376, 16)      160       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 200, 376, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 99, 187, 8)        1160      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 99, 187, 8)        0     

In [145]:
# # %% Input
# original = Input(shape=input_shape)

# # Resize to have dimensions divisible by 8 (the padding)
# resized = ResizeLayer(newsize=resize_shape)(original)

# # # Wrap-around in longitude for periodic boundary conditions

# padded = PeriodicPadding2D(padd)(resized)

# # Encoding layers
# x = Conv2D(16, (3, 3), padding='same')(padded)
# x = LeakyReLU()(x)
# x = Conv2D(8, (3, 3), strides= (2,2), padding='valid')(x)
# x = LeakyReLU()(x)
# x = Conv2D(8, (3, 3), strides= (2,2), padding='valid')(x)
# x = LeakyReLU()(x)
# x = Conv2D(8, (3, 3), strides= (2,2), padding='valid')(x)
# x = LeakyReLU()(x)

In [148]:
len(model.layers)

20

In [161]:
encoder = model.layers[:9]